In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from modules import *

In [3]:
torch.cuda.is_available()

True

In [4]:
config = Config('config-Copy11.json')

In [5]:
device = 'cuda'

In [6]:
config[...]

{'name': 'noinput_repoch_dropout',
 'batchsize': 8,
 'epoch': {'start': 1, 'end': None},
 'lr': {'initial': 0.001,
  'decay_rate': 0.99,
  'decay_step': 1,
  'override': None},
 'log_period': 1,
 'test': {'name': 'noinput_repoch_dropout',
  'epoch': 431,
  'out_prefix': 'drop_sd'}}

In [7]:
class Net_NOD_BN_Skip_SuperDeep(torch.nn.Module):
    def __init__(self):
        super().__init__()
        edge_feats = 2
        
        self.p = nn.Parameter(torch.zeros(16))
        self.conv1 = NNConv(16, 32, Linear(edge_feats, 16*32), aggr='mean')
        self.conv2 = NNConv(32, 64, Linear(edge_feats, 32*64), aggr='mean')
        self.conv3 = NNConv(64, 64, Linear(edge_feats, 64*64), aggr='mean')
        self.conv4 = NNConv(64, 64, Linear(edge_feats, 64*64), aggr='mean')
        self.conv5 = NNConv(64, 64, Linear(edge_feats, 64*64), aggr='mean')
        self.conv6 = NNConv(64, 64, Linear(edge_feats, 64*64), aggr='mean')
        self.conv7 = NNConv(64, 64, Linear(edge_feats, 64*64), aggr='mean')
        self.conv8 = NNConv(64, 64, Linear(edge_feats, 64*64), aggr='mean')
        self.conv9 = NNConv(64, 64, Linear(edge_feats, 64*64), aggr='mean')
        self.conv10 = NNConv(64, 64, Linear(edge_feats, 64*64), aggr='mean')
        self.conv11 = NNConv(64, 64, Linear(edge_feats, 64*64), aggr='mean')
        self.conv12 = NNConv(64, 64, Linear(edge_feats, 64*64), aggr='mean')
        self.conv13 = NNConv(64, 32, Linear(edge_feats, 32*64), aggr='mean')
        self.conv14 = NNConv(32, 2, Linear(edge_feats, 2*32), aggr='mean')

        self.relu = nn.LeakyReLU()
        self.bn1 = BatchNorm(32)
        self.bn2 = BatchNorm(64)
        self.bn3 = BatchNorm(64)
        self.bn4 = BatchNorm(64)
        self.bn5 = BatchNorm(64)
        self.bn6 = BatchNorm(64)
        self.bn7 = BatchNorm(64)
        self.bn8 = BatchNorm(64)
        self.bn9 = BatchNorm(64)
        self.bn10 = BatchNorm(64)
        self.bn11 = BatchNorm(64)
        self.bn12 = BatchNorm(64)
        self.bn13 = BatchNorm(32)
        
        self.dp1 = Dropout(0.2)
        self.dp2 = Dropout(0.2)
        self.dp3 = Dropout(0.2)
        self.dp4 = Dropout(0.2)
        self.dp5 = Dropout(0.2)
        self.dp6 = Dropout(0.2)
        self.dp7 = Dropout(0.2)
        self.dp8 = Dropout(0.2)
        self.dp9 = Dropout(0.2)
        self.dp10 = Dropout(0.2)
        self.dp11 = Dropout(0.2)
        self.dp12 = Dropout(0.2)



#         self.conv4 = NNConv(16+32+64, 128, Linear(2, (16+32+64)*128))
#         self.conv5 = NNConv(128,2,Linear(2,128*2))
#         self.conv2 = NNConv(16,2,Linear(2,16*2))

    def forward(self, data):
        edge_index, edge_attr = data.edge_index, data.edge_attr

        x = self.p.repeat(len(data.x), 1)
        x1 = self.dp1(self.relu(self.bn1(self.conv1(x, edge_index, edge_attr))))
        x2 = self.dp2(self.relu(self.bn2(self.conv2(x1, edge_index, edge_attr))))
        x3 = self.dp3(self.relu(self.bn3(self.conv3(x2, edge_index, edge_attr))))
        x4 = self.dp4(self.relu(self.bn4(self.conv4(x3, edge_index, edge_attr))))
        
        res_x4 = x2+x4
        
        x5 = self.dp5(self.relu(self.bn5(self.conv5(res_x4, edge_index, edge_attr))))
        x6 = self.dp6(self.relu(self.bn6(self.conv6(x5, edge_index, edge_attr))))
                            
        res_x6 = res_x4+x6
                         
        x7 = self.dp7(self.relu(self.bn7(self.conv7(res_x6, edge_index, edge_attr))))
        x8 = self.dp8(self.relu(self.bn8(self.conv8(x7, edge_index, edge_attr))))
                            
        res_x8 = res_x6+x8
                         
        x9 = self.dp9(self.relu(self.bn9(self.conv9(res_x8, edge_index, edge_attr))))
        x10 = self.dp10(self.relu(self.bn10(self.conv10(x9, edge_index, edge_attr))))
                            
        res_x10 = res_x8+x10
                         
        x11 = self.dp11(self.relu(self.bn11(self.conv11(res_x10, edge_index, edge_attr))))
        x12 = self.dp12(self.relu(self.bn12(self.conv12(x11, edge_index, edge_attr))))
                            
        res_x12 = res_x10+x12
                                        
        x13 = self.relu(self.bn13(self.conv13(res_x12, edge_index, edge_attr)))
        x14 = self.conv14(x13, edge_index, edge_attr)

        return x14

In [8]:
def load_processed_data():
    if os.path.isfile('G_list.pickle') and os.path.isfile('data_list.pickle'):
        G_list = pickle.load(open('G_list.pickle', 'rb'))
        data_list = pickle.load(open('data_list.pickle', 'rb'))
    else:
        if not os.path.isfile('data_index.txt'):
            shuffle_rome('data_index.txt')
        rome = load_rome('data_index.txt')
        G_list, data_list = convert_datalist(rome)
        pickle.dump(G_list, open('G_list.pickle', 'wb'))
        pickle.dump(data_list, open('data_list.pickle', 'wb'))
    return G_list, data_list

In [9]:
G_list, data_list = load_processed_data()

In [10]:
loader = DataLoader(data_list[:10000], batch_size=config['batchsize'],shuffle=True)
loss_ep = []

In [ ]:
start_epoch = config['epoch']['start']
if start_epoch == 1:
    model = Net_NOD_BN_Skip_SuperDeep().to(device)
else:
    model = torch.load(f"../ckpt_{config['name']}/epoch_{start_epoch}.pt").to(device)
criterion = EnergyLossVectorized()
optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr']['initial'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, 
                                            step_size=config['lr']['decay_step'], 
                                            gamma=config['lr']['decay_rate'])
print("=" * 50, file=open(f"{config['name']}.log", "a"))
epoch = start_epoch
with tqdm(total=len(loader), smoothing=0) as progress:
    while True:
        if config['lr']['override'] is not None:
            optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr']['override'])
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, 
                                                        step_size=config['lr']['decay_step'], 
                                                        gamma=config['lr']['decay_rate'])
        progress.reset()
        progress.set_description(desc=f"[epoch {epoch}/{config['epoch']['end']}]")
        train_loss = train(model, criterion, optimizer, loader, data_list, device, progress)
        loss_ep.append(train_loss)
        scheduler.step()
        if epoch == 1 and config['log_period'] != 1:
            print(epoch, loss, scheduler.get_lr(), file=open(f"{config['name']}.log", "a"))
        if epoch % config['log_period'] == 0:
            torch.save(model, f"../ckpt_{config['name']}/epoch_{epoch}.pt")
            test_loss = []
            for val_idx in range(11000, len(G_list)):
                node_pos,loss = evaluate(model, data_list[val_idx],criterion,device)
                test_loss.append(loss)
            print(f'{epoch}, train: {train_loss}, val:{np.mean(test_loss)}，{scheduler.get_lr()}', file=open(f"{config['name']}.log", "a"))
        if epoch == config['epoch']['end']:
            break
        for idx in range(10000):
            data_list[idx].x = generate_randPos(data_list[idx].x.shape[0], random.randint(0, 100000000))
        epoch += 1

# Performance Testing

In [11]:
model = torch.load(f'../ckpt_{config["test"]["name"]}/epoch_{config["test"]["epoch"]}.pt', map_location=torch.device(device))
criterion = EnergyLossVectorized()
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [12]:
ground_truth = pd.read_csv('ground_truth_loss.csv', index_col=0)

In [ ]:
folder_name = f'{config["test"]["name"]}_test'
for test_idx in tqdm(list(chain(range(0, 10000, 10), range(10000, len(data_list))))):
    G_vis = G_list[test_idx]
    node_pos,loss = evaluate(model, data_list[test_idx],criterion,device)
    if 10000 <= test_idx < 11000:
        gt_loss = ground_truth.loc[test_idx][0]
        loss_ratio = (loss - gt_loss) / gt_loss
    else:
        loss_ratio = '?'
    graph_vis(G_vis, node_pos, f'{folder_name}/{config["test"]["out_prefix"]}_model_{test_idx}_{loss}_{loss_ratio}.png') 
    node_pos = nx.nx_agraph.graphviz_layout(G_vis, prog='neato')
    plt.figure()
    nx.draw(G_vis, node_pos)
    plt.savefig(f'{folder_name}/{test_idx}.png')

In [14]:
losses = []
loss_ratios = []
for test_idx in tqdm(range(10000, 11000)):
    G_vis = G_list[test_idx]
    node_pos, loss = evaluate(model, data_list[test_idx], criterion, device)
    gt_loss = ground_truth.loc[test_idx][0]
    loss_ratio = (loss - gt_loss) / gt_loss
    losses += [loss]
    loss_ratios += [loss_ratio]

In [ ]:
np.mean(node_pos),np.std(node_pos)

In [ ]:
np.mean(data_list[11000].x.numpy()),np.std(data_list[11000].x.numpy())

In [15]:
np.mean(losses), np.std(losses)

(4.9666, 4.760008302513768)

In [16]:
np.mean(loss_ratios), np.std(loss_ratios)

(-0.8557804214093896, 0.18999037869661115)

In [64]:
class EnergyLossScaled(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, p, data, scale):
        edge_attr = data.edge_attr
        # convert per-node positions to per-edge positions
        start, end, n_nodes = node2edge(p, data)
        
        start *= scale
        end *= scale
        
        start_x = start[:, 0]
        start_y = start[:, 1]
        end_x = end[:, 0]
        end_y = end[:, 1]
        
        l = edge_attr[:, 0]
        k = edge_attr[:, 1]
        
        term1 = (start_x - end_x) ** 2
        term2 = (start_y - end_y) ** 2
        term3 = l ** 2
        term4 = 2 * l * (term1 + term2).sqrt()
        energy = k / 2 * (term1 + term2 + term3 - term4)
        return energy.sum()

In [65]:
criterion_scaled = EnergyLossScaled()

In [ ]:
truth_loss = 0
pred_loss = 0
for idx in tqdm(range(10000, 11000)):
    pred, loss = evaluate(model, data_list[idx], criterion, device)
    pos_map = nx.nx_agraph.graphviz_layout(G_list[idx], prog='neato')

    pred_mean, pred_std = pred.mean(axis=0), pred.std()
    truth = np.array(list(pos_map.values()))
    truth_mean, truth_std = truth.mean(axis=0), truth.std()
    norm_truth = (truth - truth_mean) / truth_std
    scaled_truth = norm_truth * pred_std + pred_mean

    truth_loss += criterion(torch.tensor(scaled_truth), data_list[idx])
    pred_loss += criterion(torch.tensor(pred), data_list[idx])
    
truth_loss / 1000, pred_loss / 1000

In [68]:
data, G = data_list[10000], G_list[10000]
edge_attr = data.edge_attr
pred, loss = evaluate(model, data, criterion, device)
pos_map = nx.nx_agraph.graphviz_layout(G, prog='neato')
truth = np.array(list(pos_map.values()))

In [69]:
start, end, n_nodes = node2edge(torch.tensor(truth, device=device), data)
w = edge_attr[:, 1]
d = edge_attr[:, 0]

In [70]:
u2 = ((start - end) ** 2).sum(dim=1)

In [71]:
s = (w * d).sum() / (w * u2).sum()

In [72]:
criterion_scaled(torch.tensor(truth, device=device), data, s)

tensor(72.3337, device='cuda:0', dtype=torch.float64)

In [73]:
loss

4.79

In [ ]:
type(data_list[9999].x)

In [ ]:
iterations = 5
losses = []
folder_name = f'{config["test"]["name"]}_iterative_test'
for test_idx in tqdm(range(10000, len(G_list))):
    G_vis = G_list[test_idx]
    node_pos = nx.nx_agraph.graphviz_layout(G_vis, prog='neato')
    plt.figure()
    nx.draw(G_vis, node_pos)
    plt.savefig(f'{folder_name}/{test_idx}.png')
    for i in range(iterations):
        node_pos, loss = evaluate(model, data_list[test_idx], criterion, device) 
        data_list[test_idx].x = torch.tensor(node_pos,dtype=torch.float)
    losses += [loss]
    graph_vis(G_vis, node_pos, f'{folder_name}/{config["test"]["out_prefix"]}_iter_model_{test_idx}_{loss}.png')

In [ ]:
plt.xlabel("epochs")
plt.ylabel("loss")
plt.plot(loss_ep[:1000])
plt.show()

In [ ]:
np